In [1]:
import tensorflow as tf
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

import os
import stat
import math
import random
import shutil

from tqdm import tqdm

## Code Start

In [2]:
PLACEHOLDER_DIR = './Data/Placeholder/'
TRAINING_DIR = './Data/Training/'
TESTING_DIR = './Data/Testing/'
DEV_DIR = './Data/Dev/'

SPLIT_DIST = 0.1

batik_classes = os.listdir(PLACEHOLDER_DIR)

## Changing file permission

In [ ]:
for batik_class in batik_classes:
    dir = os.path.join(PLACEHOLDER_DIR, batik_class)
    dir_content = os.listdir(dir)

    for image in tqdm(dir_content, ascii = False, desc = batik_class):
        dst_dir = os.path.join(dir, image)
        os.chmod(dst_dir, stat.S_IRWXU)

----------------------------------------------
# !RUN WITH CAUTION!
## Data Splitting
 Making the DIR necessary for storing images and spreading it from PLACEHOLDER_DIR to apropriate dir

In [5]:
# MKDIR data subdir
try:
    os.mkdir(TRAINING_DIR)
    os.mkdir(TESTING_DIR)
    os.mkdir(DEV_DIR)
except:
    print('Data subdir folder already exist')

# MKDIR working subdir
for batik_class in batik_classes:
    try:
        dir = os.path.join(TRAINING_DIR, batik_class)
        os.mkdir(dir)

        dir = os.path.join(TESTING_DIR, batik_class)
        os.mkdir(dir)

        dir = os.path.join(DEV_DIR, batik_class)
        os.mkdir(dir)
    except:
        print('Working subdir folder already exist')
        break

# Actually moving things around 
for batik_class in batik_classes:
    dir = os.path.join(PLACEHOLDER_DIR, batik_class)
    dir_content = os.listdir(dir)

    # Calculating how much image for splitting
    split_dist = math.ceil(len(dir_content) * SPLIT_DIST)

    # Picking random images
    testing_images = random.choices(dir_content, k = split_dist)

    # Moving things around
    for image in tqdm(dir_content, desc = batik_class):
        # If image is picked for testing
        if image in testing_images:
            src_dir = os.path.join(dir, image)
            dst_dir = os.path.join(TESTING_DIR, batik_class)

            shutil.copy(src_dir, dst_dir)
        else:
            src_dir = os.path.join(dir, image)
            dst_dir = os.path.join(TRAINING_DIR, batik_class)

            shutil.copy(src_dir, dst_dir)

batik-bali:   8%|█████▊                                                                 | 4/49 [00:00<00:01, 26.31it/s]

Data subdir folder already exist


batik-tambal: 100%|████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 70.61it/s]


## Purge images from working dir

In [3]:
def purge_working_image(target):
    for batik_class in tqdm(batik_classes, desc = target):
        dir = os.path.join(target, batik_class)
        
        #print('Deleting ' + dir)
        shutil.rmtree(dir, ignore_errors = True)

In [4]:
purge_working_image(TRAINING_DIR)
purge_working_image(TESTING_DIR)
purge_working_image(DEV_DIR)

./Data/Dev/: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 6667.68it/s]


----------------------------------------------

## Counting total images

In [6]:
def count_batik(target):
    try:
        batik_counter = {}

        for batik_class in batik_classes:
            dir = os.path.join(target, batik_class)
            dir_content = os.listdir(dir)

            batik_counter[batik_class] = 0
            for image in dir_content:
                batik_counter[batik_class] += 1
    except:
        print('Directory Empty!')
        print()
    else:
        for i in batik_counter:
            print(i + ' = ' + str(batik_counter[i]))

        print()
        print('Total images: ' + str(sum(batik_counter.values())))

In [7]:
print('RAW IMAGES')
print('-------------------------------------')

count_batik(PLACEHOLDER_DIR)

print('TRAINING IMAGES')
print('-------------------------------------')

count_batik(TRAINING_DIR)

print('TESTING IMAGES')
print('-------------------------------------')

count_batik(TESTING_DIR)

print('DEV IMAGES')
print('-------------------------------------')

count_batik(DEV_DIR)

RAW IMAGES
-------------------------------------
batik-bali = 49
batik-betawi = 43
batik-celup = 27
batik-cendrawasih = 25
batik-ceplok = 30
batik-ciamis = 34
batik-garutan = 31
batik-gentongan = 36
batik-kawung = 40
batik-keraton = 36
batik-lasem = 46
batik-megamendung = 46
batik-parang = 45
batik-pekalongan = 47
batik-priangan = 38
batik-sekar = 24
batik-sidoluhur = 45
batik-sidomukti = 34
batik-sogan = 23
batik-tambal = 31

Total images: 730
TRAINING IMAGES
-------------------------------------
batik-bali = 44
batik-betawi = 39
batik-celup = 24
batik-cendrawasih = 22
batik-ceplok = 27
batik-ciamis = 30
batik-garutan = 27
batik-gentongan = 32
batik-kawung = 36
batik-keraton = 32
batik-lasem = 42
batik-megamendung = 41
batik-parang = 40
batik-pekalongan = 42
batik-priangan = 34
batik-sekar = 21
batik-sidoluhur = 40
batik-sidomukti = 30
batik-sogan = 20
batik-tambal = 27

Total images: 650
TESTING IMAGES
-------------------------------------
batik-bali = 5
batik-betawi = 4
batik-celup 

## Data Augmentation

In [8]:
training_datagen = ImageDataGenerator(
    # Might need further augmentation
    rescale=1/255.0
)

testing_datagen = ImageDataGenerator(
    rescale=1/255.0
)

training_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(256,256),
    class_mode='categorical',
    batch_size=32
)

testing_generator = testing_datagen.flow_from_directory(
    TESTING_DIR,
    target_size=(256,256),
    class_mode='categorical',
    batch_size=32
)

Found 650 images belonging to 20 classes.
Found 80 images belonging to 20 classes.


## Model

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    #20 Batik class, might add more later?
    tf.keras.layers.Dense(20, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 246016)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               62980352  
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5

## Compiling

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## Fitting

In [11]:
history = model.fit(
    training_generator,
    validation_data=testing_generator,
    epochs=10
)

Epoch 1/10
21/21 [==============================] - 20s 729ms/step - loss: 9.2383 - accuracy: 0.0569 - val_loss: 2.9742 - val_accuracy: 0.0750
Epoch 2/10
21/21 [==============================] - 12s 555ms/step - loss: 2.8311 - accuracy: 0.1600 - val_loss: 3.0446 - val_accuracy: 0.1250
Epoch 3/10
21/21 [==============================] - 12s 563ms/step - loss: 2.3003 - accuracy: 0.3662 - val_loss: 3.0021 - val_accuracy: 0.1875
Epoch 4/10
21/21 [==============================] - 13s 629ms/step - loss: 1.4847 - accuracy: 0.6831 - val_loss: 3.3716 - val_accuracy: 0.0875
Epoch 5/10
21/21 [==============================] - 12s 570ms/step - loss: 0.9303 - accuracy: 0.8231 - val_loss: 3.7454 - val_accuracy: 0.1375
Epoch 6/10
21/21 [==============================] - 13s 615ms/step - loss: 0.6278 - accuracy: 0.8892 - val_loss: 3.8039 - val_accuracy: 0.1750
Epoch 7/10
21/21 [==============================] - 12s 578ms/step - loss: 0.5412 - accuracy: 0.9200 - val_loss: 3.8817 - val_accuracy: 0.1250